In [ ]:
#Set the data_root as the root folder of you unprocessed and processed data is in and expected to placed.
#Set unprocessed_folder as the folder containing all the links details.
#Set the mapped_links_file as the file which is processed by links_arround_precip.py
#Processed file will be set to [data_root]/Processed/[unprocessed_folder].csv

In [26]:
data_root = "/home/gayan/Desktop/CMLARE/Data"
#unprocessed_folder = "2018-05-08 to 2018-05-15"#folder path for the directory to be processed
unprocessed_folder = "test"
processed_folder = data_root+"/Processed/selected/"+unprocessed_folder #Output folder for each file
mapped_links_file= data_root+"/Master Files"+"/mapped_links_precip.csv"  #List of the selected links

processed_file = data_root+"/Processed/"+unprocessed_folder + ".csv"
rootUnprocessed_folder = data_root+"/Un Processed/"
unprocessed_folder = rootUnprocessed_folder + unprocessed_folder

In [27]:
import os
from shutil import copy
import csv
import geopy.distance
from math import log
from decimal import Decimal

In [28]:
link_data = []
mapped_links = []
#test_file = processed_folder+"/PM_IG30028_15_201803180000_01.csv"


mapped_links_field_names = ["cell_tower_id1","cell_tower_id2","source1_name","source2_name", "district1","district2",
                            "latitude1","latitude2","longitude1","longitude2","nearest_precip_station","distance",
                            "nearest_precip_lat","nearest_precip_long","id_1","id_2","PathLength","frequency1","frequency2","frequency_band"]

link_data_field_names = ["DeviceID","DeviceName","ResourceID","ResourceName","CollectionTime","GranularityPeriod","RSL_MAX","RSL_MIN","RSL_AVG","RSL_CUR","TLHTT","TLLTT","TSL_MAX","TSL_MIN","TSL_AVG","TSL_CUR","RLHTT","RLLTT","ATPC_N_ADJUST","ATPC_P_ADJUST","ODU_SSV_TH"]

processed_data_all = []
processed_file_field_names = ["ID","DateTime","PrecipStation","PRmax","PRmin","PTmin","PTmax","PRAvg","PTAvg","PAttAvg","XStart","YStart","RSL_MIN","RSL_MAX","RSL_AVG","TSL_MIN","TSL_MAX","TSL_AVG","SLAttn","XEnd","YEnd","PathLength","distance","Frequency","FrequencyBand"]

In [29]:
def dBm_to_W(dBm):
    power = 10 ** ((float(dBm)-30)/10)
    return Decimal(power).to_eng_string()


def W_to_dBm(power):
    dBm = 30 + 10 * log(float(power), 10)
    return (dBm)

In [30]:
directory = os.listdir(unprocessed_folder)
correct_files = []
for file in directory:
    splitted_name = file.split("_")[1]      
    if(splitted_name.endswith("28")):
        correct_files.append(file)
for file in correct_files:
    if not os.path.exists(processed_folder):
        os.makedirs(processed_folder)
    copy(src=unprocessed_folder+"/"+file,dst=processed_folder)

In [31]:
def start_mapping(files):
    global link_data,mapped_links_file,mapped_links_field_names,mapped_links,link_data_field_names,processed_data_all
    with open(mapped_links_file,"r") as mapped_links_csv:
        mapped_links = csv.DictReader(mapped_links_csv, fieldnames=mapped_links_field_names)
        next(mapped_links)                                                                                     #Skipping the Header
        mapped_links = list(mapped_links)
        print(" Reading Cell towers locations successfull")
        for file in files:
            with open(file,"r") as file_csv:
                print("processing file : "+file)
                link_data = csv.DictReader(file_csv,fieldnames=link_data_field_names)
                next(link_data)
                next(link_data)
                link_data = list(link_data)
                mapped_links_copy = mapped_links.copy()
                for link in link_data:
                    link_resource_name = link["ResourceName"]
                    processed_data = {}
                    processed_data["PRmin"] = dBm_to_W(link["RSL_MIN"])
                    processed_data["PRmax"] = dBm_to_W(link["RSL_MAX"])
                    processed_data["PRAvg"] = dBm_to_W(link["RSL_AVG"])
                    processed_data["PTmin"] = dBm_to_W(link["TSL_MIN"])
                    processed_data["PTmax"] = dBm_to_W(link["TSL_MAX"])
                    processed_data["PTAvg"] = dBm_to_W(link["TSL_AVG"])
                    processed_data["RSL_MIN"] = link["RSL_MIN"]
                    processed_data["RSL_MAX"] = link["RSL_MAX"]
                    processed_data["RSL_AVG"] = link["RSL_AVG"]
                    processed_data["TSL_MIN"] = link["TSL_MIN"]
                    processed_data["TSL_MAX"] = link["TSL_MAX"]
                    processed_data["TSL_AVG"] = link["TSL_AVG"]
                    processed_data["DateTime"] = link["CollectionTime"]                  #.replace("-","").replace(" ","").replace(":","")[:-2]
                    for mapped_link in mapped_links_copy:
                        mapped_link_resource_name1 = mapped_link["source1_name"]
                        mapped_link_resource_name2 = mapped_link["source2_name"]
                        if(link_resource_name == mapped_link_resource_name1):
                            processed_data["XStart"] = mapped_link["latitude2"]
                            processed_data["YStart"] = mapped_link["longitude2"]
                            processed_data["XEnd"] = mapped_link["latitude1"]
                            processed_data["YEnd"] = mapped_link["longitude1"]
                            processed_data["ID"] = mapped_link["id_1"]
                            processed_data["Frequency"] = mapped_link["frequency2"] 
                            processed_data["FrequencyBand"] = mapped_link["frequency_band"]
                            processed_data["distance"] = mapped_link["distance"]
                            processed_data["PrecipStation"] = mapped_link["nearest_precip_station"]
                            start_cord = (processed_data["XStart"],processed_data["YStart"])
                            end_cord = (processed_data["XEnd"],processed_data["YEnd"])
                            processed_data["PathLength"] = geopy.distance.vincenty(start_cord, end_cord).km
                            
                            transmitted_power = processed_data["PTAvg"]
                            link2 = list(filter(lambda link: link['ResourceName'] == mapped_link_resource_name2, link_data))
#                             print(link2)
                            if(len(link2)==1):
                                link2 = link2[0]
                                RSL_AVG = link2["RSL_AVG"]
                                recieved_power = dBm_to_W(RSL_AVG)
                                PAttAvg = Decimal(transmitted_power) - Decimal(recieved_power)
                                if(PAttAvg >0):
                                    processed_data["PAttAvg"] = PAttAvg
                                    processed_data["SLAttn"] = float(link["TSL_AVG"]) - float(RSL_AVG)
                            mapped_links_copy.remove(mapped_link)
                            break
                        elif(link_resource_name == mapped_link_resource_name2):
                            processed_data["XStart"] = mapped_link["latitude1"]
                            processed_data["YStart"] = mapped_link["longitude1"]
                            processed_data["XEnd"] = mapped_link["latitude2"]
                            processed_data["YEnd"] = mapped_link["longitude2"]
                            processed_data["ID"] = mapped_link["id_2"]
                            processed_data["Frequency"] = mapped_link["frequency1"] 
                            processed_data["FrequencyBand"] = mapped_link["frequency_band"]
                            processed_data["distance"] = mapped_link["distance"]
                            processed_data["PrecipStation"] = mapped_link["nearest_precip_station"]
                            processed_data["PathLength"] = mapped_link["PathLength"]
                            transmitted_power = processed_data["PTAvg"]
                            link2 = list(filter(lambda link: link['ResourceName'] == mapped_link_resource_name1, link_data))
                            if(len(link2)==1):
                                link2 = link2[0]
#                             print(link2)
                                RSL_AVG = link2["RSL_AVG"]
                                recieved_power = dBm_to_W(RSL_AVG)
                                PAttAvg = Decimal(transmitted_power) - Decimal(recieved_power)
                                if(PAttAvg >0):
                                    processed_data["PAttAvg"] = PAttAvg
                                    processed_data["SLAttn"] = float(link["TSL_AVG"]) - float(RSL_AVG)
                            mapped_links_copy.remove(mapped_link)
                            break
                    if(("XStart" in processed_data) and ("YStart" in processed_data) and ("XEnd" in processed_data) and ("YEnd" in processed_data) and ("PAttAvg" in processed_data)):
                        processed_data_all.append(processed_data)
                print("processing file : "+file+" successfull")
                    
def process_all():
    directory = os.listdir(processed_folder)
    directory = list(directory)
    correct_file_paths = []
    for file in directory:
#         print(file)
        correct_file_paths.append(str(processed_folder)+"/"+file)
#     print(directory)
    start_mapping(correct_file_paths)
    print("process Successfull")

process_all()

 Reading Cell towers locations successfull
processing file : /home/gayan/Desktop/CMLARE/Data/Processed/selected/test/PM_IG30028_15_201805051500_01.csv
-36.5
-36.0
-32.7
-34.0
-34.1
-38.8
-32.9
-33.7
-33.5
-32.9
-31.9
-33.6
-37.9
-34.0
-34.4
-32.1
-31.7
-34.3
-40.0
-33.3
-32.9
-30.9
-32.8
-36.5
-42.1
-31.0
-31.8
-33.9
-31.2
-31.6
-31.8
-31.5
-31.1
-32.7
-34.8
-39.9
-38.9
-33.1
-32.9
-32.5
-32.0
-33.5
-31.4
-33.9
-39.0
-34.8
-30.3
-30.9
-43.2
-34.8
-39.2
-32.9
-33.1
-34.0
-33.7
-33.7
-33.6
-34.2
-41.0
-31.2
-35.9
-34.8
-31.3
-30.6
-29.8
-33.8
-31.8
-31.6
-31.8
-32.6
-29.0
-27.8
-33.1
-29.8
-34.5
-34.7
-34.3
-31.8
-36.9
-35.0
-40.3
-33.1
-34.3
-32.7
-32.4
-32.5
-37.4
-32.9
-32.8
-31.0
-33.7
-39.5
-31.5
-36.7
-30.5
-30.7
-32.3
-32.2
-32.7
-37.1
-32.0
-32.0
-31.1
-33.6
-32.7
-34.5
-32.3
-33.9
-28.7
-28.3
-32.5
-31.7
-30.7
-35.5
-32.8
-29.9
-33.0
-34.4
-33.1
-38.2
-28.2
-28.6
-32.0
-31.6
-32.4
-35.3
-36.3
-30.9
-33.9
-35.8
-35.6
-30.2
-30.3
-30.3
-34.1
-33.3
-33.8
-33.4
-34.1
-34.2
-29.5
-34

-34.4
-36.2
-36.2
-31.9
-29.4
-28.9
-35.1
-30.1
-32.6
-33.4
-35.6
-39.7
-44.3
-33.4
-38.1
-30.1
-34.3
-29.5
-34.4
-29.5
-34.4
-31.6
-32.3
-33.8
-38.0
-38.6
-43.8
-30.2
-35.1
-34.5
-38.7
-30.9
-31.2
-41.1
-40.5
-31.8
-29.8
-32.3
-31.7
-31.6
-31.3
-30.2
-30.2
-33.3
-33.8
-33.4
-32.4
-31.4
-36.7
-22.9
-32.0
-32.4
-33.5
-34.5
-35.2
-33.5
-27.8
-26.7
-31.3
-38.8
-30.6
-31.3
-30.0
-33.2
-34.9
-36.1
-39.6
-35.3
-29.9
-36.7
-41.3
-32.7
-37.7
-34.1
-30.8
-32.2
-32.6
-30.2
-33.6
-34.1
-28.6
-33.4
-31.9
-30.7
-35.0
-34.5
-32.4
-32.3
-38.0
-37.5
-35.0
-36.1
-31.2
-31.4
-33.1
-30.7
-33.2
-34.6
-35.9
-31.6
-30.6
-31.9
-31.3
-38.8
-41.1
-35.2
-40.3
-31.1
-35.5
-31.9
-33.1
-30.8
-32.5
-33.5
-29.5
-32.8
-31.6
-32.0
-32.9
-33.8
-35.1
-33.7
-34.8
-28.2
-27.8
-31.1
-35.9
-36.6
-35.4
-34.3
-31.8
-32.0
-31.5
-33.7
-30.8
-31.5
-37.0
-37.0
-33.2
-32.8
-42.9
-41.3
-32.9
-32.9
-32.6
-37.9
-36.1
-30.5
-34.7
-33.4
-33.0
-31.5
-31.3
-31.1
-32.1
-31.3
-33.9
-34.6
-31.2
-32.2
-36.4
-31.2
-34.9
-27.9
-28.5
-36.0
-29.

-29.7
-32.0
-30.5
-32.0
-36.2
-35.7
-35.0
-35.0
-34.3
-34.9
-32.6
-31.1
-31.0
-29.7
-32.5
-36.0
-31.2
-31.3
-32.3
-31.2
-39.8
-43.3
-32.5
-38.3
-32.1
-32.5
-29.2
-29.2
-34.6
-32.9
-30.3
-30.8
-29.4
-33.1
-33.1
-32.2
-31.3
-36.3
-32.5
-37.6
-31.0
-30.9
-24.1
-22.7
-31.5
-37.3
-32.5
-39.3
-38.1
-29.4
-26.9
-31.9
-34.5
-38.6
-30.3
-34.3
-31.8
-31.7
-32.7
-32.8
-30.3
-30.3
-31.8
-31.4
-29.7
-24.6
-34.4
-34.2
-31.8
-36.9
-32.2
-31.5
-28.2
-33.5
-30.8
-30.6
-31.6
-36.6
-30.9
-30.7
-33.9
-34.8
-30.5
-32.5
-33.2
-34.6
-38.9
-31.2
-36.4
-32.4
-36.4
-34.6
-30.3
-35.9
-35.3
-29.0
-34.2
-35.7
-33.2
-37.9
-32.2
-38.3
-35.7
-32.7
-32.6
-32.5
-31.1
-31.5
-31.1
-30.8
-35.7
-33.8
-38.5
-22.0
-32.6
-32.2
-31.2
-31.4
-36.1
-29.0
-32.1
-30.4
-29.9
-31.6
-31.1
-31.0
-36.8
-36.3
-35.2
-34.4
-34.5
-33.4
-34.5
-30.6
-35.6
-34.1
-32.4
-21.1
-26.0
-30.4
-34.2
-31.7
-34.3
-32.2
-37.8
-31.1
-32.3
-37.4
-33.0
-34.1
-34.5
-39.5
-32.0
-31.3
-31.8
-32.3
-33.4
-28.4
-29.0
-28.8
-29.3
-35.7
-31.2
-31.7
-40.6
-39.8
-33.

-30.1
-30.0
-36.7
-30.9
-31.1
-31.4
-32.0
-37.1
-33.9
-37.9
-33.2
-33.4
-33.5
-28.5
-29.5
-29.9
-31.9
-32.0
-35.4
-36.5
-32.4
-31.7
-33.2
-31.4
-36.2
-30.3
-31.7
-35.5
-35.2
-37.7
-42.2
-46.8
-33.1
-32.3
-32.0
-33.6
-33.8
-33.6
-31.5
-36.6
-34.8
-30.8
-31.8
-31.6
-31.7
-34.9
-48.7
-30.6
-31.9
-36.6
-34.7
-35.7
-40.7
-43.3
-31.5
-31.0
-33.8
-33.6
-32.3
-33.3
-33.3
-32.0
-32.3
-30.5
-30.5
-32.8
-30.8
-29.4
-38.7
-39.2
-30.4
-30.8
-31.6
-35.1
-35.1
-31.5
-31.3
-41.2
-42.0
-29.9
-29.5
-35.7
-41.4
-30.4
-28.6
-32.8
-38.2
-35.0
-38.0
-39.2
-30.9
-31.3
-33.9
-34.1
-32.4
-31.4
-34.6
-35.0
-34.0
-30.1
-31.4
-29.6
-33.5
-37.4
-33.8
-35.4
-33.3
-36.1
-41.6
-31.2
-31.8
-30.9
-34.7
-40.8
-31.1
-32.4
-35.9
-33.0
-34.6
-34.4
-33.2
-32.5
-24.6
-33.1
-33.1
-34.8
-33.3
-28.1
-32.4
-32.6
-33.0
-31.1
-31.5
-34.8
-33.8
-33.7
-37.3
-29.8
-35.6
-41.1
-37.3
-32.0
-31.8
-35.0
-39.1
-31.8
-39.4
-36.3
-34.7
-32.7
-32.8
-33.1
-32.3
-33.5
-32.7
-32.2
-38.0
-33.5
-38.5
-33.9
-34.7
-34.1
-39.9
-28.4
-29.1
-30.5
-30.

-35.4
-32.0
-37.2
-34.8
-33.2
-34.8
-31.2
-30.9
-35.2
-35.7
-32.4
-30.6
-30.8
-32.7
-32.4
-29.1
-33.2
-34.5
-33.9
-30.2
-33.7
-33.0
-32.8
-33.8
-32.3
-32.0
-32.8
-29.5
-32.4
-34.2
-38.8
-31.0
-41.1
-33.2
-30.7
-35.7
-33.6
-28.6
-31.9
-31.2
-32.9
-31.6
-30.9
-34.0
-28.8
-29.3
-32.1
-32.1
-31.8
-36.0
-33.7
-32.5
-35.5
-35.7
-35.3
-35.0
-29.7
-35.2
-31.1
-30.9
-32.4
-31.9
-35.9
-34.0
-33.9
-36.2
-37.2
-38.6
-28.1
-28.6
-30.9
-34.3
-38.9
-33.3
-38.0
-31.3
-31.3
-32.0
-31.7
-32.4
-31.7
-31.3
-30.6
-32.4
-32.8
-29.8
-29.7
-31.9
-32.2
-34.6
-33.7
-28.8
-31.6
-34.6
-33.0
-31.8
-36.9
-31.4
-30.4
-29.6
-29.8
-31.7
-30.6
-31.2
-30.6
-32.0
-34.3
-38.4
-30.2
-33.7
-34.3
-30.1
-34.1
-34.6
-34.2
-33.0
-32.9
-32.6
-51.3
-51.1
-30.5
-30.6
-33.3
-34.7
-36.3
-32.8
-37.4
-33.6
-34.1
-31.5
-30.7
-31.2
-28.2
-30.6
-30.9
-32.9
-32.9
-31.1
-36.8
-31.2
-32.1
-30.9
-35.8
-33.4
-30.0
-34.2
-30.6
-57.2
-56.9
-35.3
-34.4
-36.4
-35.7
-31.2
-31.9
-30.9
-33.0
-33.6
-32.9
-31.1
-30.5
-32.8
-31.8
-32.1
-32.7
-32.4
-33.

...

-37.9
-33.1
-30.3
-30.9
-35.1
-33.1
-32.7
-30.8
-35.3
-35.2
-32.6
-33.0
-30.6
-31.6
-34.6
-35.2
-29.4
-34.3
-30.3
-30.9
-32.8
-37.3
-32.6
-33.7
-31.5
-31.2
-34.4
-34.1
-28.4
-27.9
-33.9
-32.5
-33.8
-35.0
-31.8
-31.3
-33.3
-32.8
-31.6
-31.4
-47.7
-33.0
-38.0
-34.4
-32.7
-32.8
-32.6
-32.7
-30.1
-29.0
-29.5
-33.7
-34.9
-33.0
-32.6
-33.6
-31.8
-31.9
-31.8
-30.2
-29.4
-29.3
-31.3
-30.5
-34.5
-27.6
-36.7
-32.6
-31.9
-30.9
-35.5
-57.6
-62.5
-31.0
-34.6
-34.7
-29.8
-30.2
-30.5
-31.6
-36.5
-31.6
-31.0
-35.8
-31.3
-32.0
-27.8
-28.3
-34.7
-34.8
-31.0
-32.5
-29.5
-28.6
-33.5
-32.2
-36.1
-35.2
-36.5
-31.0
-29.2
-31.9
-33.7
-33.2
-37.1
-34.7
-33.9
-32.0
-32.4
-33.2
-37.5
-35.3
-35.0
-30.8
-33.9
-33.3
-31.8
-32.4
-31.0
-31.7
-30.6
-38.7
-34.1
-30.0
-34.8
-40.6
-26.3
-36.2
-36.3
-32.4
-42.6
-41.9
-31.2
-30.8
-32.5
-31.4
-31.6
-31.3
-31.3
-31.1
-31.3
-30.8
-39.3
-40.2
-35.9
-32.0
-29.8
-32.3
-33.1
-32.9
-34.9
-32.5
-42.7
-42.3
-32.5
-32.0
-32.3
-30.7
-33.6
-35.5
-29.5
-35.7
-36.3
-40.7
-33.1
-37.9
-32.

-39.6
-31.2
-34.8
-28.5
-32.7
-34.1
-36.2
-32.9
-31.2
-31.6
-29.9
-28.2
-33.5
-33.7
-28.1
-30.6
-30.7
-30.7
-32.7
-37.0
-39.4
-25.6
-29.4
-29.3
-34.5
-32.4
-32.2
-33.3
-34.3
-30.8
-30.0
-29.0
-31.4
-32.5
-36.8
-32.3
-34.1
-31.5
-30.2
-31.6
-32.1
-31.3
-33.1
-31.2
-33.1
-32.3
-32.0
-34.8
-34.6
-33.2
-31.3
-33.7
-33.2
-33.4
-35.8
-31.4
-31.4
-32.8
-32.6
-36.0
-32.3
-27.7
-32.5
-32.5
-32.6
-37.2
-35.6
-34.6
-35.1
-35.6
-34.9
-33.9
-40.0
-38.1
-31.5
-37.8
-31.8
-30.5
-30.7
-31.3
-36.9
-42.3
-31.7
-31.8
-36.9
-31.6
-33.1
-32.7
-32.5
-33.1
-34.2
-39.9
-34.4
-37.9
-37.1
-31.8
-28.6
-31.4
-31.6
-30.7
-30.6
-31.5
-35.1
-32.8
-34.1
-34.3
-34.5
-27.4
-32.4
-34.6
-34.9
-30.6
-31.1
-31.4
-35.7
-33.6
-32.2
-36.4
-43.2
-48.1
-32.8
-30.3
-30.2
-31.3
-26.2
-25.5
-36.2
-33.3
-31.8
-35.7
-31.4
-33.1
-29.3
-28.6
-29.8
-30.4
-34.4
-33.6
-42.0
-37.9
-39.5
-31.3
-31.0
-30.5
-30.7
-36.4
-35.7
-31.7
-33.1
-33.2
-32.6
-34.1
-32.5
-36.9
-31.1
-31.4
-30.9
-31.3
-31.2
-31.2
-32.0
-30.5
-32.2
-37.8
-32.3
-35.5
-32.

-29.7
-28.6
-33.6
-30.5
-32.6
-33.9
-34.5
-31.2
-31.7
-35.4
-35.4
-34.3
-37.6
-41.6
-34.4
-34.6
-34.3
-46.0
-51.1
-33.4
-33.3
-37.9
-33.8
-35.8
-36.7
-35.9
-33.8
-34.6
-35.9
-41.3
-34.9
-31.6
-36.5
-35.4
-35.2
-35.6
-35.5
-29.9
-34.8
-30.9
-35.9
-28.2
-28.0
-31.8
-32.1
-34.5
-29.7
-36.2
-35.5
-31.6
-31.7
-34.1
-33.5
-35.8
-30.7
-36.4
-33.6
-33.0
-32.0
-31.8
-34.5
-31.8
-33.0
-33.8
-30.4
-30.9
-32.7
-33.9
-31.8
-30.2
-38.3
-28.3
-28.3
-35.0
-31.2
-31.3
-32.1
-33.5
-31.6
-32.3
-32.1
-33.9
-39.2
-34.1
-31.7
-30.1
-31.9
-44.2
-37.2
-44.6
-31.5
-32.5
-33.2
-36.5
-35.8
-31.5
-32.0
-31.8
-32.3
-25.7
-31.1
-34.4
-37.6
-32.1
-27.8
-42.2
-43.3
-36.9
-38.2
-34.1
-33.5
-33.5
-31.6
-30.6
-31.8
-33.6
-30.3
-28.8
-32.7
-34.8
-34.9
-32.9
-36.5
-30.7
-35.2
-33.2
-32.6
-31.8
-33.6
-35.3
-32.1
-30.8
-30.6
-32.1
-28.7
-32.7
-34.9
-40.3
-32.2
-31.2
-36.0
-36.0
-40.6
-35.8
-32.0
-31.0
-33.8
-29.0
-33.6
-26.8
-28.1
-35.8
-40.8
-32.8
-29.7
-36.3
-31.8
-32.9
-30.1
-34.1
-43.3
-35.1
-35.3
-34.8
-35.7
-31.3
-32.

-34.1
-33.3
-33.0
-32.2
-31.1
-30.8
-31.1
-36.7
-33.7
-33.6
-31.9
-29.1
-33.1
-34.5
-29.4
-29.0
-32.1
-41.7
-32.9
-37.6
-47.2
-34.8
-38.4
-33.4
-33.1
-31.3
-33.9
-35.7
-40.7
-31.5
-31.1
-31.4
-30.9
-34.9
-41.2
-31.4
-30.1
-29.2
-39.6
-39.1
-32.0
-36.8
-32.1
-31.6
-29.4
-36.8
-37.2
-31.5
-37.0
-34.1
-33.2
-29.7
-33.5
-34.1
-37.8
-28.1
-28.0
-34.5
-35.3
-36.0
-34.6
-37.1
-39.3
-31.1
-29.4
-42.2
-31.7
-30.3
-35.6
-31.2
-30.3
-32.2
-30.7
-30.3
-30.6
-35.5
-35.1
-35.0
-40.3
-32.2
-33.7
-34.0
-33.2
-33.1
-38.0
-31.7
-33.3
-36.4
-32.9
-36.7
-31.8
-31.0
-29.2
-29.0
-34.4
-33.0
-31.9
-31.5
-31.6
-31.0
-31.0
-35.5
-30.8
-35.4
-35.5
-31.4
-31.4
-36.4
-29.7
-30.4
-31.5
-34.8
-40.7
-30.3
-33.4
-36.9
-41.6
-31.5
-31.3
-38.8
-38.7
-27.5
-32.4
-33.1
-32.2
-32.3
-32.7
-32.4
-32.6
-31.8
-29.9
-32.8
-31.8
-45.6
-45.7
-33.9
-35.3
-34.0
-35.1
-33.2
-30.5
-36.3
-33.6
-33.1
-32.8
-31.9
-36.3
-26.4
-26.4
-37.0
-38.2
-34.1
-32.1
-31.5
-31.2
-29.6
-29.8
-35.2
-33.8
-33.3
-32.3
-32.1
-33.5
-33.2
-33.2
-32.0
-45.

-34.0
-32.3
-37.7
-31.1
-32.3
-31.5
-30.3
-29.8
-36.5
-32.6
-36.8
-37.1
-35.4
-34.4
-36.9
-30.7
-32.3
-29.7
-34.6
-34.3
-34.8
-33.0
-33.4
-33.8
-31.8
-31.8
-33.1
-39.3
-37.2
-41.2
-28.0
-31.0
-33.7
-33.6
-33.6
-34.2
-35.3
-37.8
-43.4
-33.2
-31.7
-36.9
-31.2
-32.2
-31.9
-32.2
-31.5
-31.1
-31.5
-31.1
-34.7
-36.2
-35.9
-40.3
-33.7
-38.2
-30.7
-31.9
-33.4
-32.7
-29.4
-30.1
-34.1
-39.1
-37.6
-42.3
-32.9
-32.8
-31.1
-45.0
-34.4
-40.2
-32.1
-33.5
-34.2
-34.7
-31.4
-31.8
-31.6
-29.5
-29.4
-25.1
-31.1
-33.8
-41.7
-46.9
-32.9
-33.8
-35.7
-32.0
-33.2
-35.3
-36.3
-37.1
-31.3
-30.4
-33.7
-32.9
-30.3
-31.0
-33.5
-32.0
-37.6
-31.5
-33.6
-38.9
-34.6
-39.7
-30.5
-35.3
-37.7
-41.8
-30.2
-30.1
-33.5
-38.6
-37.6
-36.0
-33.0
-30.8
-33.0
-31.5
-35.3
-35.8
-32.6
-32.4
-32.4
-34.9
-30.5
-36.3
-30.6
-32.4
-40.9
-40.9
-30.1
-30.9
-33.9
-32.6
-38.2
-31.0
-31.5
-30.9
-31.9
-31.5
-31.6
-26.7
-31.5
-42.7
-38.7
-43.9
-35.0
-33.5
-34.1
-39.3
-29.9
-29.9
-38.8
-30.9
-42.9
-32.9
-33.2
-33.0
-29.4
-29.8
-31.2
-36.1
-34.

In [32]:
def write_processed_files():
    global processed_data_all
    print("Writing to file.....")
    with open(processed_file,"w") as processed_file_csv:
        dictionary_writer = csv.DictWriter(processed_file_csv,processed_file_field_names)
        dictionary_writer.writeheader()
        processed_data_all = sorted(processed_data_all , key=lambda k: k['DateTime'])
#         print(processed_data_all[0].keys())
        dictionary_writer.writerows(processed_data_all)
    print("Writing to file Finished..")
write_processed_files()

Writing to file.....
Writing to file Finished..
